In [1]:
# Analysis of flow cytometry data for the CAR generations dataset

In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
from math import log2

from helpers import mpl_plotting_helpers as mph
from helpers import stats_helpers as sh
from helpers import general_helpers as gh

In [80]:
conditions = {"DMSO" : [0,1,2],
              "pp1" : [3,4,5],
              "soq" : [6,7,8],
              "u0126" : [9,10,11],
              "tpi" : [12,13,14],
              "n22" : [15,16,17],
              "pv" : [18,19,20]}

In [36]:
# First, need to merge all the excel files within a directory and only keep the channel of interest (APC)
def make_aggregate(file_list, 
                   filetype = ".csv", 
                   take_col = "APC-A",
                   newfile_name = None):
    cols = []
    colnames = []
    for f in file_list:
        colnames.append(f.split("/")[-1])
        if filetype == ".csv":
            desired_col = list(pd.read_csv(f)[take_col].astype(float).to_numpy())
            cols.append(gh.transform_values(desired_col, transform = float) )
        else:
            desired_col = list(pd.read_excel(f)[take_col].astype(float).to_numpy())
            cols.append(gh.transform_values(desired_col, transform = float))
    if newfile_name != None:
        df = pd.DataFrame(cols, columns = colnames).write_excel(newfile_name)
    return cols, colnames

In [72]:
nocc_f_cd69 = {"JE6" : glob.glob("je6_nocc/je6_nocc_cd3_cd69/*.csv"),
               "J132": glob.glob("j132_nocc/j132_nocc_gfp_cd69/*.csv"),
               "J133": glob.glob("j133_nocc/j133_nocc_gfp_cd69/*.csv"),
               "J134": glob.glob("j134_nocc/j134_nocc_gfp_cd69/*.csv"),
               "J135": glob.glob("j135_nocc/j135_nocc_gfp_cd69/*.csv")}

nocc_f_gfp = {"JE6" : glob.glob("je6_nocc/je6_nocc_cd3/*.csv"),
              "J132": glob.glob("j132_nocc/j132_nocc_gfp/*.csv"),
              "J133": glob.glob("j133_nocc/j133_nocc_gfp/*.csv"),
              "J134": glob.glob("j134_nocc/j134_nocc_gfp/*.csv"),
              "J135": glob.glob("j135_nocc/j135_nocc_gfp/*.csv")}

nocc_d_cd69 = {key : make_aggregate(value) for key, value in nocc_f_cd69.items()}
nocc_d_gfp = {key : make_aggregate(value) for key, value in nocc_f_gfp.items()}

nocc_perc = {key : gh.transpose(*[[nocc_d_cd69[key][1][i], len(nocc_d_cd69[key][0][i])/len(nocc_d_gfp[key][0][i])*100] for i in range(len(nocc_d_cd69[key][0]))])
             for key, value in nocc_d_cd69.items()}
nocc_mfi = {key : gh.transpose(*[[nocc_d_cd69[key][1][i], sh.mean(nocc_d_cd69[key][0][i])] for i in range(len(nocc_d_cd69[key][0]))])
             for key, value in nocc_d_cd69.items()}



In [127]:
stim_f_cd69 = {"JE6" : glob.glob("je6_stim/je6_stim_cd3_cd69/*.csv"),
               "J132": glob.glob("j132_stim/j132_stim_gfp_cd69/*.csv"),
               "J133": glob.glob("j133_stim/j133_stim_gfp_cd69/*.csv"),
               "J134": glob.glob("j134_stim/j134_stim_gfp_cd69/*.csv"),
               "J135": glob.glob("j135_stim/j135_stim_gfp_cd69/*.csv")}

stim_f_gfp = {"JE6" : glob.glob("je6_stim/je6_stim_cd3/*.csv"),
              "J132": glob.glob("j132_stim/j132_stim_gfp/*.csv"),
              "J133": glob.glob("j133_stim/j133_stim_gfp/*.csv"),
              "J134": glob.glob("j134_stim/j134_stim_gfp/*.csv"),
              "J135": glob.glob("j135_stim/j135_stim_gfp/*.csv")}

stim_d_cd69 = {key : make_aggregate(value) for key, value in stim_f_cd69.items()}
stim_d_gfp = {key : make_aggregate(value) for key, value in stim_f_gfp.items()}

stim_perc = {key : gh.transpose(*[[stim_d_cd69[key][1][i], len(stim_d_cd69[key][0][i])/len(stim_d_gfp[key][0][i])*100] for i in range(len(stim_d_cd69[key][0]))])
             for key, value in stim_d_cd69.items()}
#stim_perc = {key : [gh.replace_value(row, 0.0, float('nan')) for row in value] for key, value in stim_perc.items()}
stim_mfi = {key : gh.transpose(*[[stim_d_cd69[key][1][i], sh.mean(stim_d_cd69[key][0][i])] for i in range(len(stim_d_cd69[key][0]))])
             for key, value in stim_d_cd69.items()}

In [248]:
# We has the data, so now what comparisons do I want to show?
### First, I want Unstim for each condition, and Stim for each condition

def _logical_ignore_comps(labelled_line_groups,
                          group_strs,
                          xgroup_strs):
    """
    Only want to compare along a line group (e.g. timecourse) or
    down an x-column (e.g. JE6 DMSO 0m vs JE6 U0126 0m), but not
    all the random other comparisons because statistically they're
    kind of useless
    
    So this function will find all of the pairs that are useless
    """
    groups_unpacked = []
    for group in labelled_line_groups:
        groups_unpacked += group
    # This will hold the ignored pairs
    ignore_me_senpai = []
    # First, get all pairs
    paired = gh.make_pairs(groups_unpacked,
                           dupes = False,
                           reverse = False)
    # Then iterate over and check the labels
    for p in paired:
        gs_check = 0
        xs_check = 0
        # Check all the group strings
        for gs in group_strs:
            if gs_check == 1:
                pass
            elif gs in p[0][0] and gs in p[1][0]:
                gs_check = 1
        # Check all the xgroup strings
        for xs in xgroup_strs:
            if xs_check == 1:
                pass
            elif xs in p[0][0] and xs in p[1][0]:
                xs_check = 1
        # If there isn't a match, in either, ignore
        if gs_check == 0 and xs_check == 0:
            ignore_me_senpai.append(p)
    # Return the ignored pairs at the end
    return ignore_me_senpai

def within_group_comp(metric_dict, condition_dict, comparison,
                      statout = {}, # stattype = sh.HolmSidak, outnum = 2,
                      ymin = 0,
                      ymax = None,
                      ymax_nudge = 5,
                      figsize = (3,6),
                      filename = "default",
                      colours = ["white", "grey", "hotpink", "cyan", "mediumpurple"],
                      label = "CD69+ Percentage",
                      fontsize = 8):
    # basically want to grab all the data and pass it to mph.dotplot
    # plus do some slutty stats on it
    # First, grab all the data
    data = [[key, [value[1][i] for i in condition_dict[comparison]]] for key, value in metric_dict.items()]
    if ymax == None:
        all_data = gh.unpack_list([d[1] for d in data if d[1] == d[1]])
        all_data = max([item for item in all_data if item == item])
        ymax = round(all_data, ndigits = -1) + ymax_nudge
    # Then pass this into the stattype object 
    #stats = stattype(*data, override = True)
    #stats.write_output(f"{filename}")
    mph.dotplot(data, rotation = 90, comparisons = statout,
               colours= colours, ylabel = label,
                title = fr"{comparison}",
                filename = f"{filename}.pdf",
                    ymin = ymin,
                    ymax = ymax,
                    figsize = figsize,
                    markersize = 50,
                    comp_fontsize = fontsize,
                    tick_fontsize = fontsize,
                    label_fontsize = fontsize,
                    title_fontsize = fontsize)
    return None

def stim_unstim_comp(unstim_dict,
                     stim_dict,
                     comparison,
                     condition_dict,
                     group_strs_ig = [],
                     xgroups_ig = [],
                     stimstr = "+ Raji",
                     unstr = "- Raji",
                     #stattype = sh.HolmSidak,outnum = 2,
                     ymin = 0,
                     ymax = None,
                     ymax_nudge = 5,
                     figsize = (3,6),
                     filename = "default",
                     colours = ["white", "white",
                                "grey", "grey",
                                "hotpink", "hotpink",
                                "cyan", "cyan",
                                "mediumpurple", "mediumpurple"],
                     label = "% CD69+",
                     fontsize = 8):
    # Want to plot the stim/unstim for each condition, all cell lines
    # So first, grab the data for each stim for the given condition
    stim_d = [[f"{key}{stimstr}", [value[1][i] for i in condition_dict[comparison]]] for key, value in stim_dict.items()]
    un_d = [[f"{key}{unstr}", [value[1][i] for i in condition_dict[comparison]]] for key, value in unstim_dict.items()]
    data_ig = []
    data = []
    for i in range(len(stim_d)):
        data_ig.append([un_d[i],stim_d[i]])
        data.append(un_d[i])
        data.append(stim_d[i])
    if ymax == None:
        all_data = gh.unpack_list([d[1] for d in data if d[1] == d[1]])
        all_data = max([item for item in all_data if item == item])
        ymax = round(all_data, ndigits = -1) + ymax_nudge
    # Then proceed as with the previous onegithub
    #if group_strs_ig != [] and xgroups_ig != []:
    #    ignore = _logical_ignore_comps(data_ig,
    #                                   group_strs = group_strs_ig,
    #                                   xgroup_strs = xgroups_ig)
    #else:
    #    ignore = []
    #stats = stattype(*data, override = True, no_comp = ignore)
    #stats.write_output(f"{filename}")
    mph.dotplot(data, rotation = 90,
               colours= colours, ylabel = label,
                title = fr"{comparison}",
                filename = f"{filename}.pdf",
                    ymin = ymin,
                    ymax = ymax,
                    figsize = figsize,
                    markersize = 50,
                    comp_fontsize = fontsize,
                    tick_fontsize = fontsize,
                    label_fontsize = fontsize,
                    title_fontsize = fontsize)
    return None

def within_cell_comp(metric_dict, 
                     condition_dict, 
                     conditions, # list in this case
                     cell_line,
                      statout = {}, # stattype = sh.HolmSidak, outnum = 2,
                      ymin = 0,
                      ymax = None,
                      ymax_nudge = 5,
                      figsize = (3,6),
                      filename = "default",
                      colours = ["white", "grey", "hotpink", "cyan", "mediumpurple"],
                      label = "CD69+ Percentage",
                      fontsize = 8):
    # basically want to grab all the data and pass it to mph.dotplot
    # First, grab all the data
    data = []
    for cond in conditions:
        for key, value in metric_dict.items():
            if key == cell_line:
                data.append([f"{cond}", [value[1][i] for i in condition_dict[cond]]])
    if ymax == None:
        all_data = gh.unpack_list([d[1] for d in data if d[1] == d[1]])
        all_data = max([item for item in all_data if item == item])
        ymax = round(all_data, ndigits = -1) + ymax_nudge
    # Then pass this into the stattype object 
    #stats = stattype(*data, override = True)
    #stats.write_output(f"{filename}")
    mph.dotplot(data, rotation = 90, comparisons = statout,
               colours= colours, ylabel = label,
                title = fr"{cell_line}",
                filename = f"{filename}.pdf",
                    ymin = ymin,
                    ymax = ymax,
                    figsize = figsize,
                    markersize = 50,
                    comp_fontsize = fontsize,
                    tick_fontsize = fontsize,
                    label_fontsize = fontsize,
                    title_fontsize = fontsize)
    return None


In [263]:
# Do all the binary comparisons and correct like it's proteomics
data = []
for cell, measurements in nocc_perc.items():
    for inhibitor, indices in conditions.items():
        data.append([f"{cell} {inhibitor} - Raji", [measurements[1][i] for i in indices]])
for cell, measurements in stim_perc.items():
    for inhibitor, indices in conditions.items():
        data.append([f"{cell} {inhibitor} + Raji", [measurements[1][i] for i in indices]])

global_test = sh.HolmSidak(*data)
global_test.write_output("figs/combined_stats_perc")

FileExistsError: [Errno 17] File exists: 'figs/combined_stats_perc.csv'

In [261]:
# Do all the binary comparisons and correct like it's proteomics
mfi_data = []
for cell, measurements in nocc_mfi.items():
    for inhibitor, indices in conditions.items():
        mfi_data.append([f"{cell} {inhibitor} - Raji", [measurements[1][i] for i in indices]])
for cell, measurements in stim_mfi.items():
    for inhibitor, indices in conditions.items():
        mfi_data.append([f"{cell} {inhibitor} + Raji", [measurements[1][i] for i in indices]])

global_test = sh.HolmSidak(*mfi_data)
global_test.write_output("figs/combined_stats_mfi")

'File figs/combined_stats_mfi.csv.csv has been written.'

In [219]:
len([p for p in global_test.output[2]["pvalue"] if p < 0.05])

1787

In [239]:
up_filenames = {key:f"figs/unstim/perc_{key}" for key, value in conditions.items()}

for key, value in conditions.items():
    within_group_comp(nocc_perc,
                  conditions,
                  key,
                      statout = global_test.output[2],
                      label = fr"% CD69$+$",
                      ymin = -4,
                  ymax = 95,
                    filename = up_filenames[key])

Too many comparisons provided. Filtering by significance : 0.05
Too many comparisons provided. Filtering by significance : 0.05
Too many comparisons provided. Filtering by significance : 0.05
Too many comparisons provided. Filtering by significance : 0.05
Too many comparisons provided. Filtering by significance : 0.05
Too many comparisons provided. Filtering by significance : 0.05
Too many comparisons provided. Filtering by significance : 0.05


In [240]:
um_filenames = {key:f"figs/unstim/mfi_{key}" for key, value in conditions.items()}

for key, value in conditions.items():
    within_group_comp(nocc_mfi,
                  conditions,
                  key,
                      label = "CD69+ MFI",
                  ymax = None,
                      ymax_nudge = 10000,
                    filename = um_filenames[key])

In [242]:
sp_filenames = {key:f"figs/stim/perc_{key}" for key, value in conditions.items()}

for key, value in conditions.items():
    within_group_comp(stim_perc,
                  conditions,
                  key,
                      label = fr"% CD69$+$",
                      ymin = -4,
                  ymax = 95,
                    filename = sp_filenames[key])

In [243]:
sm_filenames = {key:f"figs/stim/mfi_{key}" for key, value in conditions.items()}

for key, value in conditions.items():
    within_group_comp(stim_mfi,
                  conditions,
                  key,
                      label = "CD69+ MFI",
                  ymax = None,
                    ymax_nudge = 10000,
                    filename = sm_filenames[key])

In [244]:
su_filenames = {key:f"figs/stim_v_unstim/{key}_perc" for key, value in conditions.items()}

for key, value in conditions.items():
    stim_unstim_comp(nocc_perc,
                     stim_perc,
                     key,
                     conditions,
                     group_strs_ig = ["JE6", "J132", "J133", "J134", "J135"],
                     xgroups_ig = ["- Raji", "+ Raji"],
                     stimstr = "+ Raji",
                     unstr = "- Raji",
                     ymin = -4,
                     ymax = 95,
                     ymax_nudge = 5,
                     figsize = (6,6),
                     filename = su_filenames[key],
                     colours = ["white", "white",
                                "grey", "grey",
                                "hotpink", "hotpink",
                                "cyan", "cyan",
                                "mediumpurple", "mediumpurple"],
                     label = "% CD69+")

In [256]:
uc_filenames = {}
for key, value in nocc_perc.items():
    temp_names = []
    for g in ["kinases", "phosphatases"]:
        temp_names.append(f"./figs/cells/{key}_{g}_unstim")
    uc_filenames[key] = temp_names

uc_groups = [["DMSO", "pp1", "soq", "u0126"],
             ["DMSO", "n22", "tpi", "pv"],]

glob_colours = ["white", "grey", "hotpink", "cyan", "mediumpurple"]
cell_cs = {}
i=0
for key, value in nocc_perc.items():
    cell_cs[key] = [glob_colours[i] for _ in range(4)]
    i+=1

for key, value in nocc_perc.items():
    for i in range(2):
        if i== 0:
            within_cell_comp(nocc_perc, 
                     conditions, 
                     uc_groups[i], # list in this case
                     key,
                      statout = {}, # stattype = sh.HolmSidak, outnum = 2,
                      ymin = -4,
                      ymax = 25,
                      ymax_nudge = 5,
                      figsize = (3,6),
                      filename = uc_filenames[key][i],
                      colours = cell_cs[key],
                      label = "% CD69+",
                      fontsize = 12)
        if i==1:
            within_cell_comp(nocc_perc, 
                     conditions, 
                     uc_groups[i], # list in this case
                     key,
                      statout = {}, # stattype = sh.HolmSidak, outnum = 2,
                      ymin = -4,
                      ymax = 95,
                      ymax_nudge = 5,
                      figsize = (3,6),
                      filename = uc_filenames[key][i],
                      colours = cell_cs[key],
                      label = "% CD69+",
                      fontsize = 12)

In [259]:
sc_filenames = {}
for key, value in stim_perc.items():
    temp_names = []
    for g in ["kinases", "phosphatases"]:
        temp_names.append(f"./figs/cells/{key}_{g}_stim")
    sc_filenames[key] = temp_names

for key, value in stim_perc.items():
    for i in range(2):
        if i== 0:
            within_cell_comp(stim_perc, 
                     conditions, 
                     uc_groups[i], # list in this case
                     key,
                      statout = {}, # stattype = sh.HolmSidak, outnum = 2,
                      ymin = -4,
                      ymax = 95,
                      ymax_nudge = 5,
                      figsize = (3,6),
                      filename = sc_filenames[key][i],
                      colours = cell_cs[key],
                      label = "% CD69+",
                      fontsize = 12)
        if i==1:
            within_cell_comp(stim_perc, 
                     conditions, 
                     uc_groups[i], # list in this case
                     key,
                      statout = {}, # stattype = sh.HolmSidak, outnum = 2,
                      ymin = -4,
                      ymax = 95,
                      ymax_nudge = 5,
                      figsize = (3,6),
                      filename = sc_filenames[key][i],
                      colours = cell_cs[key],
                      label = "% CD69+",
                      fontsize = 12)

In [272]:
comps_of_interest_1 = [("JE6 DMSO - Raji", "J132 DMSO - Raji"),
                    ("JE6 DMSO - Raji", "J133 DMSO - Raji"),
                    ("JE6 DMSO - Raji", "J134 DMSO - Raji"),
                    ("JE6 DMSO - Raji", "J135 DMSO - Raji"),
                    ("JE6 DMSO + Raji", "J132 DMSO + Raji"),
                    ("JE6 DMSO + Raji", "J133 DMSO + Raji"),
                    ("JE6 DMSO + Raji", "J134 DMSO + Raji"),
                    ("JE6 DMSO + Raji", "J135 DMSO + Raji")]
comps_of_interest_2 =[("JE6 DMSO - Raji", "JE6 n22 - Raji"),
                    ("JE6 DMSO - Raji", "JE6 tpi - Raji"),
                    ("JE6 DMSO - Raji", "JE6 pv - Raji"),
                    ("J132 DMSO - Raji", "J132 n22 - Raji"),
                    ("J132 DMSO - Raji", "J132 tpi - Raji"),
                    ("J132 DMSO - Raji", "J132 pv - Raji"),
                    ("J133 DMSO - Raji", "J133 n22 - Raji"),
                    ("J133 DMSO - Raji", "J133 tpi - Raji"),
                    ("J133 DMSO - Raji", "J133 pv - Raji"),
                    ("J134 DMSO - Raji", "J134 n22 - Raji"),
                    ("J134 DMSO - Raji", "J134 tpi - Raji"),
                    ("J134 DMSO - Raji", "J134 pv - Raji"),
                    ("J135 DMSO - Raji", "J135 n22 - Raji"),
                    ("J135 DMSO - Raji", "J135 tpi - Raji"),
                    ("J135 DMSO - Raji", "J135 pv - Raji"),
                     
                    ("JE6 DMSO + Raji", "JE6 n22 + Raji"),
                    ("JE6 DMSO + Raji", "JE6 tpi + Raji"),
                    ("JE6 DMSO + Raji", "JE6 pv + Raji"),
                    ("J132 DMSO + Raji", "J132 n22 + Raji"),
                    ("J132 DMSO + Raji", "J132 tpi + Raji"),
                    ("J132 DMSO + Raji", "J132 pv + Raji"),
                    ("J133 DMSO + Raji", "J133 n22 + Raji"),
                    ("J133 DMSO + Raji", "J133 tpi + Raji"),
                    ("J133 DMSO + Raji", "J133 pv + Raji"),
                    ("J134 DMSO + Raji", "J134 n22 + Raji"),
                    ("J134 DMSO + Raji", "J134 tpi + Raji"),
                    ("J134 DMSO + Raji", "J134 pv + Raji"),
                    ("J135 DMSO + Raji", "J135 n22 + Raji"),
                    ("J135 DMSO + Raji", "J135 tpi + Raji"),
                    ("J135 DMSO + Raji", "J135 pv + Raji"),
                     
                    ("JE6 DMSO - Raji", "JE6 pp1 - Raji"),
                    ("JE6 DMSO - Raji", "JE6 soq - Raji"),
                    ("JE6 DMSO - Raji", "JE6 u0126 - Raji"),
                    ("J132 DMSO - Raji", "J132 pp1 - Raji"),
                    ("J132 DMSO - Raji", "J132 soq - Raji"),
                    ("J132 DMSO - Raji", "J132 u0126 - Raji"),
                    ("J133 DMSO - Raji", "J133 pp1 - Raji"),
                    ("J133 DMSO - Raji", "J133 soq - Raji"),
                    ("J133 DMSO - Raji", "J133 u0126 - Raji"),
                    ("J134 DMSO - Raji", "J134 pp1 - Raji"),
                    ("J134 DMSO - Raji", "J134 soq - Raji"),
                    ("J134 DMSO - Raji", "J134 u0126 - Raji"),
                    ("J135 DMSO - Raji", "J135 pp1 - Raji"),
                    ("J135 DMSO - Raji", "J135 soq - Raji"),
                    ("J135 DMSO - Raji", "J135 u0126 - Raji"),
                     
                    ("JE6 DMSO + Raji", "JE6 pp1 + Raji"),
                    ("JE6 DMSO + Raji", "JE6 soq + Raji"),
                    ("JE6 DMSO + Raji", "JE6 u0126 + Raji"),
                    ("J132 DMSO + Raji", "J132 pp1 + Raji"),
                    ("J132 DMSO + Raji", "J132 soq + Raji"),
                    ("J132 DMSO + Raji", "J132 u0126 + Raji"),
                    ("J133 DMSO + Raji", "J133 pp1 + Raji"),
                    ("J133 DMSO + Raji", "J133 soq + Raji"),
                    ("J133 DMSO + Raji", "J133 u0126 + Raji"),
                    ("J134 DMSO + Raji", "J134 pp1 + Raji"),
                    ("J134 DMSO + Raji", "J134 soq + Raji"),
                    ("J134 DMSO + Raji", "J134 u0126 + Raji"),
                    ("J135 DMSO + Raji", "J135 pp1 + Raji"),
                    ("J135 DMSO + Raji", "J135 soq + Raji"),
                    ("J135 DMSO + Raji", "J135 u0126 + Raji"),]

In [274]:
stats = gh.transpose(*[value for key, value in global_test.output[2].items() if key in ["Group 1", "Group 2", "pvalue"]])

for v in stats:
    if (v[0], v[1]) in comps_of_interest_1 or (v[1], v[0]) in comps_of_interest_1:
        print(f"{v[0]:^30} vs {v[1]:^30} pvalue: {v[2]:^40}")

       JE6 DMSO - Raji         vs        J132 DMSO - Raji        pvalue:           0.006201290680580285          
       JE6 DMSO - Raji         vs        J133 DMSO - Raji        pvalue:           9.58948914984159e-05          
       JE6 DMSO - Raji         vs        J134 DMSO - Raji        pvalue:           0.06234740041286735           
       JE6 DMSO - Raji         vs        J135 DMSO - Raji        pvalue:            0.9748403532446576           
       JE6 DMSO + Raji         vs        J132 DMSO + Raji        pvalue:                   0.0                   
       JE6 DMSO + Raji         vs        J133 DMSO + Raji        pvalue:                   0.0                   
       JE6 DMSO + Raji         vs        J134 DMSO + Raji        pvalue:                   0.0                   
       JE6 DMSO + Raji         vs        J135 DMSO + Raji        pvalue:                   0.0                   


In [273]:
for v in stats:
    if (v[0], v[1]) in comps_of_interest_2 or (v[1], v[0]) in comps_of_interest_2:
        print(f"{v[0]:^30} vs {v[1]:^30} pvalue: {v[2]:^40}")

       JE6 DMSO - Raji         vs         JE6 pp1 - Raji         pvalue:            0.9896244246308827           
       JE6 DMSO - Raji         vs         JE6 soq - Raji         pvalue:            0.9999967885724058           
       JE6 DMSO - Raji         vs        JE6 u0126 - Raji        pvalue:            0.9999946173650228           
       JE6 DMSO - Raji         vs         JE6 tpi - Raji         pvalue:                   1.0                   
       JE6 DMSO - Raji         vs         JE6 n22 - Raji         pvalue:                   1.0                   
       JE6 DMSO - Raji         vs         JE6 pv - Raji          pvalue:                   0.0                   
       J132 DMSO - Raji        vs        J132 pp1 - Raji         pvalue:           1.67081239299538e-06          
       J132 DMSO - Raji        vs        J132 soq - Raji         pvalue:           0.001060445039341551          
       J132 DMSO - Raji        vs       J132 u0126 - Raji        pvalue:           0.108

In [267]:
stats[0]

['JE6 DMSO - Raji', 'JE6 pp1 - Raji', 0.9896244246308827]